In [25]:
%load_ext autoreload
%autoreload 2

import os
from entities.simulation import Simulation
from model.TrafficMDP import TrafficMDP

# dictionary with preferences for the simulation:
# - name: name of the simulation
# - car_spawn_frequency: frequency of car spawn in seconds
# - max_simulation_time: maximum time of the simulation in seconds
# - traffic_manage_rule: can be traffic mdp or fixed time
# - traffic_mdp: instance of the class TrafficMDP
# - fixed_time: dictionary with the fixed time for each traffic light


"""simulations = [
     {
          "name": "MDP Test",
          "car_spawn_frequency": 1.5,
          "max_simulation_time": 120,
          "traffic_manage_rule": "traffic mdp",
          "traffic_mdp": TrafficMDP(),
     },
     {
          "name": "Fixed Time Test",
          "car_spawn_frequency": 1.5,
          "max_simulation_time": 120,
          "traffic_manage_rule": "fixed time",
     },
]"""


# Define a dictionary with preferences car direction spawn:
# - direction: direction of the car spawn
# - spawn_frequency: frequency of car spawn in seconds
# - car_spawn_probability: probability of car spawn
# - car_spawn_time: time of the car spawn
# - car_spawn_time_variance: variance of the time of the car spawn
# - car_spawn_time_min: minimum time of the car spawn
# - car_spawn_time_max: maximum time of the car spawn
# - car_spawn_time_variance_min: minimum variance of the time of the car spawn

car_spawn_preferences = [
     {
          "direction": "NS",
          "spawn_frequency": 1.5,
          "spawn_times": [{"time_start": 0, "time_end": 30}, {"time_start": 60, "time_end": 90}],
     },
     {
          "direction": "WE",
          "spawn_frequency": 1.5,
          "spawn_times": [{"time_start": 30, "time_end": 60},]
     },
     {
          "direction": "both",
          "spawn_frequency": 1.5,
          "spawn_times": [{"time_start": 0, "time_end": 30}, {"time_start": 60, "time_end": 90}],
     }

]

traffic_mdp = TrafficMDP()

simulation = Simulation(
     name="Ambient Test",
     car_spawn_frequency=1.5,  # seconds
     max_simulation_time=120,  # seconds
)

simulation.run(traffic_mdp)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Simulation ended.
